As Análises realizadas neste notebook estão descritas em mais detalhes e comentadas nos dois posts do medium abaixo:

[Analisando Cotas Parlamentares: Uma Jornada de 2010 a 2020 (pt. 1) ](https://medium.com/@samuevan/analisando-cotas-parlamentares-uma-jornada-de-2010-a-2020-pt-1-3c72dbf906eb)

[Analisando Cotas Parlamentares: Uma Jornada de 2010 a 2020 (pt. 2) ](https://medium.com/@samuevan/analisando-cotas-parlamentares-uma-jornada-de-2010-a-2020-pt-2-a5eb6a35d257)

Para simplificar a execução destes notebooks no Kaggle eu gerei um dataset com as bases de dados. Contudo, caso você deseje rodar localmente e baixar os arquivos atualizados é só baixar o notebook e remover os comentários das linhas de importação do serenata_toolbox

In [ ]:
import pandas as pd
import numpy as np
import glob
import os

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})
import seaborn as sn
import seaborn as sns
from plotly.graph_objs import *
#import chart_studio
#import chart_studio.plotly as py
#import plotly.graph_objects as go
import plotly.express as px

#Para o Kaggle eu deixei comentado pois já subi os arquivos direto para um dataset
#from serenata_toolbox.federal_senate.dataset import Dataset as SenateDataset
#from serenata_toolbox.chamber_of_deputies.reimbursements import Reimbursements as ChamberDataset


from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Download e processamentos iniciais

In [ ]:
PATH_CHAMBER_DATA = "../input/reimbursemensts-deputies-brazil"
DOWNLOAD_FILES = False
INIT_YEAR = 2008
FINAL_YEAR = 2021

In [ ]:
if DOWNLOAD_FILES:
    #Para os dados da câmara é preciso informar o ano que desejamos baixar
    for year in range(INIT_YEAR,FINAL_YEAR+1):        
        chamber = ChamberDataset(year, PATH_CHAMBER_DATA)
        try:
            chamber()
        except:
            continue

<blockquote>Após baixar os dados todos eles são carregados para um mesmo dataframe. 
Primeiro listamos todos os arquivos reimbursement*.csv usando o glob
e depois carregamos todos os arquivos listados usando list comprehension.
Por fim, os dataframes carregados são empilhamos/concatenamos usando pd.concat</blockquote>

In [ ]:
#lista arquivos de interesse
files_chamber = glob.glob(os.path.join(PATH_CHAMBER_DATA,"reimbursement*.csv"))        
#carrega e concatena todos os arquivos num único dataframe
df_chamber = pd.concat([pd.read_csv(file_chamber
                                    ,sep=','
                                    ,low_memory=False) 
                        for file_chamber in files_chamber])

#como eu fiz um concat dos dataframes, cada um manteve o seu indice original
#Dessa forma, preciso resetar os indices para que as linhas mantenham indices unicos
df_chamber.reset_index(drop=True, inplace=True)
df_chamber.head()

In [ ]:
#Contagem de registros por ano
df_chamber.year.value_counts()

>Um dos processamentos do serenata-toolbox é traduzir as descrições das subcotas para inglês. Apesar disso, o processamento do serenata ainda mantém os arquivos originais com as descrições em portguguês. Como o artigo vai ser escrito em português vou utilizar as descrições dos arquivos originais

In [ ]:
#Foi preciso pegar vários anos para que todas as descrições aparecessem na base
df_chamber_ptbr = pd.concat([pd.read_csv(os.path.join(PATH_CHAMBER_DATA,'Ano-2020.csv'),sep=';')
                            ,pd.read_csv(os.path.join(PATH_CHAMBER_DATA,'Ano-2014.csv'),sep=';')
                            ,pd.read_csv(os.path.join(PATH_CHAMBER_DATA,'Ano-2008.csv'),sep=';')])


#mantive apenas as colunas de interesse, removi todas as duplicatas e transformei num dicionario
df_chamber_ptbr = df_chamber_ptbr[['numSubCota','txtDescricao']].drop_duplicates().sort_values(by='numSubCota')
desc_port = df_chamber_ptbr.set_index('numSubCota')['txtDescricao'].to_dict()

#Por fim, usei o dicionario para substituir o conteúdo da coluna subquota_description
df_chamber['subquota_description'] = df_chamber['subquota_number'].apply(lambda x : desc_port[x])
del desc_port, df_chamber_ptbr

In [ ]:
df_chamber.columns

In [ ]:
VIEWS_COLS = ['congressperson_name' ,'subquota_description' ,'document_value'
                ,'remark_value' ,'total_net_value' ,'issue_date' ,'month' ,'year']

# Limpeza inicial dos dados

Aqui vamos fazer uma limpeza e pré processamento inicial dos dados. Em uma análise preliminar encontrei uma série de problemas e vou realizar as seguintes correções.
<ol>
    <li>Remover registros onde existe incosistencia de reembolso e glosa </li>
    <li>Remover dos registros com valores negativos</li>
    <li>Remover registros sem partido e sem estado</li>
    <li>Dropar 2008 pq tem muitos NANs</li>
    <li>Agrupar os nomes de partidos que se alteraram (tb posso checar pelo numero)</li>    
    <li>Remover registros de 2008, 2009 e 2021</li>
    <ol>
            <li>PMDB virou MDB</li>
            <li>PEN virou PATRIOTAS (Se fundiu com o PRP em 2018 e deixou de usar a sigla PATRI em 2019)</li>
            <li>PP e PP** estao se referindo ao mesmo partido</li>        
            <li>PPS virou CIDADANIA </li>
            <li>PRB virou REPUBLICANOS (verificar deputados REPUBLICANOS antes do mandato de 2018)</li>
            <li>PR virou PL</li>
    </ol>
</ol>

## Removendo registros onde existe incosistencia de reembolso e glosa

>A Glosa é um valor da nota fiscal em questão que não será reembolsada. O valor real reembolsado deve ser o valor da nota menos o valor da glosa. Como notei que existem várias inconsistencias nesses dados, realizo a seguinte operação: Quando para um registro, o valor da nota (document_value) menos a glosa (remark_value) tem uma diferença de mais de 10 centavos para o valor total da nota (total_net_value) eu removo o registo em questão.

In [ ]:
#Crio uma coluna nova para essa controlar o motivo da remoção dos registros
df_chamber['removal_reason'] = np.zeros(df_chamber.shape[0], dtype=np.int8)

#constroi a regra de remocao
removal_rule = abs((df_chamber.document_value-df_chamber.remark_value)-df_chamber.total_net_value) >= 0.1
#marca no dataset inicial os registros removidos por esta regra
df_chamber.loc[removal_rule, 'removal_reason'] = 1

## Removendo registros com valores negativos

In [ ]:
#2) remover os valores negativos
removal_rule = (df_chamber.removal_reason == 0) & (df_chamber.total_net_value < 0)
df_chamber.loc[removal_rule, 'removal_reason'] = 2

## Removendo registros sem partido e sem estado

> Parte dos registros está sem a informação de partido. Alguns destes são relativos a gastos de liderança, nesse caso vou separá-los para análise futura. Contudo, existem outros que são de fato congressistas ('FERNANDO ESTIMA', 'LUIZ MOREIRA', 'PROFESSOR LUIZINHO') e estão sem essa informação. Nestes casos estou removendo os registros

In [ ]:
#Gerando o dataframe das lideranças
removal_rule = ((df_chamber.removal_reason == 0) 
                & (df_chamber.party.isna()) 
                & (df_chamber.congressperson_name.str.contains('LID')))
df_chamber.loc[removal_rule, 'removal_reason'] = 3

df_chamber_clean_lid = df_chamber[removal_rule]

df_chamber.loc[df_chamber.removal_reason == 3].shape

In [ ]:
#como só temos esses tres deputados com registros sem partido eu vou dropar todos 
#e criar um dataset separado para liderancas

removal_rule = (df_chamber.removal_reason == 0) & (df_chamber.party.isna())
df_chamber.loc[removal_rule, 'removal_reason'] = 4
df_chamber.loc[df_chamber.removal_reason == 4].shape


In [ ]:
#Checando todas as descrições de subcotas
#As descrições das cotas podem ser encontradas em 
#https://www.camara.leg.br/cota-parlamentar/2016_Glossario_para_o_CotasNet.pdf
df_chamber.subquota_description.unique()

## Remover registros de 2008, 2009 e 2021

In [ ]:
removal_rule = (df_chamber.removal_reason == 0) \
                & ((df_chamber.year == 2008) \
                   | (df_chamber.year == 2009) \
                   | (df_chamber.year == 2021))
df_chamber.loc[removal_rule, 'removal_reason'] = 5
df_chamber.loc[df_chamber.removal_reason == 5].shape

### Dataset Final

In [ ]:
df_chamber_clean = df_chamber.loc[df_chamber.removal_reason == 0].copy()

In [ ]:
print("Tamanho do Dataset original {}".format(df_chamber.shape[0]))

print("Registros removidos por inconsistencia no valor de reembolso/glosa: {} ({:.2f}% do total)".format(
     df_chamber[df_chamber.removal_reason == 1].shape[0]
    , 100*(df_chamber[df_chamber.removal_reason == 1].shape[0]/df_chamber.shape[0])))

print("Registros removidos por valor negativo de reembolso/glosa: {} ({:.2f}% do total)".format(
     df_chamber[df_chamber.removal_reason == 2].shape[0]
    , 100*(df_chamber[df_chamber.removal_reason == 2].shape[0]/df_chamber.shape[0])))

print("Registros removidos por não possuir informacao de partido: {} ({:.2f}% do total)".format(
     df_chamber[df_chamber.removal_reason.isin([3,4])].shape[0]
    , 100*(df_chamber[df_chamber.removal_reason.isin([3,4])].shape[0]/df_chamber.shape[0])))


print("Registros removidos referentes ao ano de 2008: {} ({:.2f}% do total)".format(
     df_chamber[df_chamber.removal_reason == 5].shape[0]
    , 100*(df_chamber[df_chamber.removal_reason == 5].shape[0]/df_chamber.shape[0])))

print("Tamanho do Dataset Limpo {} ({:.2f}% do dataset original)".format(
        df_chamber_clean.shape[0],100*(df_chamber_clean.shape[0]/df_chamber.shape[0])))

print("Tamanho do Dataset de Lideres {} ({:.2f}% do dataset original)".format(
        df_chamber_clean_lid.shape[0],100*(df_chamber_clean_lid.shape[0]/df_chamber.shape[0])))

In [ ]:
del df_chamber

## Agrupar os nomes de partidos que se alteraram

In [ ]:
#Links relevantes para a análise
#https://www.camara.leg.br/deputados/quem-sao
#https://www.camara.leg.br/cota-parlamentar/index.jsp
#https://www2.camara.leg.br/transparencia/cota-para-exercicio-da-atividade-parlamentar/dados-abertos-cota-parlamentar
#https://www2.camara.leg.br/transparencia/dados-abertos/dados-abertos-legislativo
#https://www2.camara.leg.br/transparencia/cota-para-exercicio-da-atividade-parlamentar/explicacoes-sobre-o-formato-dos-arquivos-xml
#https://www.camara.leg.br/deputados/bancada-na-eleicao


##### PMDB/MBD

In [ ]:
#Checar os deputados do MDB antes de 2018
df_chamber_clean[(df_chamber_clean.party=='MDB') & (df_chamber_clean.year < 2014)]

In [ ]:
#Setando todos os deputados do MDB/PMDB para usarem o mesmo nome
df_chamber_clean.loc[df_chamber_clean.party == 'PMDB', 'party'] = 'MDB'

#### PEN/PRP/PATRI/PATRIORAS

In [ ]:
df_chamber_clean[(df_chamber_clean.party.isin(['PEN','PRP','PATRI','PATRIOTA']))][['congressperson_name', 'party']]\
        .drop_duplicates().sort_values(by='congressperson_name')

In [ ]:
df_chamber_clean.loc[df_chamber_clean.party.isin(['PRP','PEN','PATRI']), 'party'] = 'PATRIOTA'

##### PP/PP**

In [ ]:
df_chamber_clean[(df_chamber_clean.party=='PP')][['congressperson_name', 'party']]\
        .drop_duplicates().sort_values(by='congressperson_name')


In [ ]:
df_chamber_clean[(df_chamber_clean.party.isin(['PP**','PP']))][['congressperson_name','party']]\
        .drop_duplicates().sort_values(by='congressperson_name')

In [ ]:
#Setando todos os deputados do PP/PP** para usarem o mesmo nome
df_chamber_clean.loc[df_chamber_clean.party == 'PP**', 'party'] = 'PP'

#### SDD/SOLIDARIEDADE

In [ ]:
#Me parece que todos os deputados que foram para o solidariedade em 2013 tiveram alteracao no banco
#e o partido dos mesmos está como SDD, mesmo antes de terem mudado de partido

df_chamber_clean[(df_chamber_clean.party.isin(['SOLIDARIEDADE','SDD']))][['congressperson_name', 'party']]\
        .drop_duplicates()\
        .sort_values(by='congressperson_name')

In [ ]:
df_chamber_clean.loc[df_chamber_clean.party == 'SDD', 'party'] = 'SOLIDARIEDADE'

#### PPS/CIDADANIA

In [ ]:
#Setando todos os deputados do PPS/CIDADANIA para usarem o mesmo nome
df_chamber_clean.loc[df_chamber_clean.party == 'PPS', 'party'] = 'CIDADANIA'

#### PRB/REPUBLICANOS

In [ ]:
df_chamber_clean[(df_chamber_clean.party.isin(['PRB','REPUBLICANOS']))][['congressperson_name', 'party']]\
        .drop_duplicates()\
        .sort_values(by='congressperson_name').head(20)

In [ ]:
#Setando todos os deputados do PRB/REPUBLICANOS para usarem o mesmo nome
df_chamber_clean.loc[df_chamber_clean.party == 'PRB', 'party'] = 'REPUBLICANOS'

#### PR/PL

In [ ]:
#Setando todos os deputados do PPS/CIDADANIA para usarem o mesmo nome
df_chamber_clean.loc[df_chamber_clean.party == 'PR', 'party'] = 'PL'

# Analises iniciais

In [ ]:
#inicialmente vamos sumarizar os dados por ano e tipo de reembolso
df_subquota_month = df_chamber_clean.groupby(by=['year','month','subquota_description']).agg(
    sum_values = pd.NamedAgg(column='total_net_value',aggfunc=sum)
    ).reset_index()


df_subquota_year = df_chamber_clean.groupby(by=['year','subquota_description']).agg(
    sum_values = pd.NamedAgg(column='total_net_value',aggfunc=sum)
    ).reset_index()

In [ ]:
df_subquota_year.head()

In [ ]:
#Ao pivotear o dataframe deixamos ele num formato onde as linhas serão os anos (ao anos e meses)
#E as coluns serão as despesas
df_subquota_month = df_subquota_month.pivot(['year','month'],'subquota_description','sum_values')
df_subquota_year = df_subquota_year.pivot('year','subquota_description','sum_values')

In [ ]:
df_subquota_year.head()

## Evolução dos gastos totais nos anos

> Nesta primeira análise esto interessado em verificar como o gasto da cota parlamentar, como um todo, variou durante os anos

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,5))
plot = sns.lineplot(x=df_subquota_year.index
            , y=df_subquota_year.sum(axis=1)/1000000 #apenas para ficar no formato de milhões
            , ax = ax 
            )

ax.set_ylim(bottom=100, top=250)
plot.set_ylabel('Tipo de reembolso (R$ Milhões)')
plot.set_xlabel('')
xticks = np.arange(df_subquota_year.index[0],df_subquota_year.index[-1]+1)
plot.set_xticks(xticks)
plot.set_xticklabels(df_subquota_year.index ,rotation=90)

In [ ]:
months_dict = {1:'Jan',2:'Fev',3:'Mai',4:'Abr',5:'Mar',6:'Jun'
            ,7:'Jul',8:'Ago',9:'Set',10:'Out',11:'Nov',12:'Dez'}

> Nesta segnda análise veremos a variação da CEAP mês a mês

In [ ]:
#soma os reembolsos por colunas (tipo do reembolso)
reinbursements_by_year = df_subquota_month.sum(axis=1)/1000000
reinbursements_by_year = reinbursements_by_year.reset_index(name='value')
#coluna para usar de identificadores de ano/mes
reinbursements_by_year['year_month'] = reinbursements_by_year[['year','month']].apply(
                                                lambda tup : f'{tup.year}_{tup.month}', axis=1)

#coluna com os nomes dos meses
reinbursements_by_year['month_str'] = reinbursements_by_year['month'].apply(
                                                lambda x : months_dict[x])

In [ ]:

fig,ax = plt.subplots(2,1,figsize=(20,9))

#plot dos gastos mes a mes
sns.lineplot(data= reinbursements_by_year
            , x='year_month'
            , y='value'
            , ax = ax[0]
            , marker="o"
            )

ax[0].set_ylabel('Total de Reembolsos (R$ Milhões)')
ax[0].set_xlabel('')

#Selecionando apenas alguns meses para mostrar na legenda do eixo x
xticks = np.arange(0,len(reinbursements_by_year.values),4)
ax[0].set_xticks(xticks)
ax[0].set_xticklabels([months_dict[i%12] for i in range(1,len(reinbursements_by_year.values),4)]
                      ,rotation=90)

#adicionando marcações ao final de cada ano
[ax[0].axvline(i*12-1, linestyle=':' ) for i in range(1,12)]
_ = plt.xticks(rotation=90)


[ax[0].annotate(text=2010+i, xy=(i*12+2, 25)
                , xycoords='data'
                , fontsize=16
                , alpha=0.8
                , color='gray'
                , style='oblique') for i in range(11)]


#plot da media dos gastos nos meses para entender a sazonalidade
sns.lineplot(data= reinbursements_by_year
            , x='month_str'
            , y='value'
            , ax = ax[1]
            )
ax[1].set_ylabel('Total de Reembolsos (R$ Milhões)')
ax[1].set_xlabel('')

> Aqi também veremos os gastos numa escala mensal, contudo vamos desagregar os dados e observar separadamente os gastos com divulgação e com passagens aéreas. Note que o dataframe que vamos passar como entrada para o gráfico é ligeiramente diferente do dataframe anterior, uma vez que de cada tipo de gastos tem a sua própria coluna

In [ ]:
cols = ['PASSAGEM AÉREA - RPA'
        ,'DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.']
reinbursements_by_year_cols = df_subquota_month[cols]/1000000
reinbursements_by_year_cols = reinbursements_by_year_cols.reset_index()
reinbursements_by_year_cols['month_str'] = reinbursements_by_year_cols['month'].apply(
                                                lambda x : months_dict[x])

reinbursements_by_year_cols

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(20,5))

sns.lineplot(data=reinbursements_by_year_cols[cols],marker="o", ax=ax)

ax.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)
ax.set_ylabel('Total de Reembolsos (R$ Milhões)')

xticks = np.arange(0,len(reinbursements_by_year.values),4)
ax.set_xticks(xticks)
ax.set_xticklabels([x for pos,x in enumerate(reinbursements_by_year_cols.month_str) if pos%4 == 0]
                      ,rotation=90)


[ax.axvline(i*12-1, linestyle=':' ) for i in range(1,12)]

[ax.annotate(text=2010+i, xy=(i*12+2, 8)
                , xycoords='data'
                , fontsize=16
                , color='gray'
                , style='oblique') for i in range(11)]

#remove o titulo da legenda
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles[0:], labels=labels[0:])

## Avaliando maiores gastos

> A seguir os gastos com as cotas serão analisados para as diferentes categorias. Inicialmente em uma perspectiva de média histórico e após isso serão realizadas comparações ano a ano.

In [ ]:

pd.set_option('display.float_format', lambda x: '%.1f' % x)
reinbursements_acc = df_subquota_year.mean()/1000000
reinbursements_2020_acc = df_subquota_year.loc[2020].fillna(0)/1000000

reinbursements_acc.sort_values(ascending=False,inplace=True)
reinbursements_acc

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,9))

sns.barplot(x=reinbursements_acc.values
            ,y=reinbursements_acc.index
            ,ax = ax
            ,color='lightblue')

ax.set_ylabel('Tipo de Reembolso')
ax.set_xlabel('Total de Reembolsos (R$ Milhões)')

_ = plt.xticks(rotation=90)

> Comparando os gastos em 2019 e 2020

In [ ]:
pd.set_option('display.float_format', lambda x: '%.1f' % x)
#diferente do caso anterior onde estávamos interessados na média anual
#aqui queremos ver os valores totais gastos nos dois anos de interesse
reinbursements_2019_2020 = df_chamber_clean.groupby(by=['year','subquota_description']).agg(
    sum_values = pd.NamedAgg(column='total_net_value'
                             ,aggfunc=lambda x : sum(x)/1000000)).loc[[2019,2020]].reset_index()

reinbursements_2019_2020.sort_values(by='sum_values', ascending=False,inplace=True)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,9))

sns.barplot(data=reinbursements_2019_2020
            ,x='sum_values'
            ,y='subquota_description'
            ,hue='year'
            ,ax = ax
            ,palette= 'Blues'
            )
ax.set_ylabel('Tipo de Reembolso')
ax.set_xlabel('Total de Reembolsos (R$ Milhões)')

_ = plt.xticks(rotation=90)
#remove o titulo da legenda
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles[0:],labels=labels[0:], bbox_to_anchor=(0.95, 0.1),borderaxespad=0 )

In [ ]:
#df_aux = df_chamber_clean[df_chamber_clean.subquota_description == 'DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.']

df_aux_parties = df_chamber_clean.groupby(by=['party','year','subquota_description']).agg(
    sum_values = pd.NamedAgg(column='total_net_value',aggfunc=sum)
    ).reset_index()

df_aux_parties = df_aux_parties.pivot(['party','year'],'subquota_description','sum_values')
df_aux_parties_percent = df_aux_parties.divide(df_aux_parties.sum(axis=1), axis=0).fillna(0).reset_index()
df_aux_parties_percent

### Avaliando os reembolsos no tempo (considerando apenas os 10 maiores gastos)

> Nesta etapa vamos analisar a evolução dos gastos de cada categoria durante os anos. Este gráfico já vai carregar uma quantidade muito grande de informação. Assim para não ficar ainda mais carregado vamos utilizar apenas os 10 maiores gastos no período analisado. Nesta visualização em específico vou utilizar o Plotly para permitir a interação com o gráfico

In [ ]:
#Dicionario com algumas abreviaturas para os nomes das categorias
label_dict = {"DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.": "Divulgação"
,"CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.": "Consultorias"
,"LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES": "Loc. Veic."
,"MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR": "Man. Escr."
,"PASSAGEM AÉREA - RPA":"Pass. Aérea - RPA"
,"PASSAGEM AÉREA - REEMBOLSO":"Pass. Aérea - Reembolso"
,"TELEFONIA":"Telefonia"  
,'SERVIÇOS POSTAIS':'Serviços - Postais'
,'COMBUSTÍVEIS E LUBRIFICANTES.':'Combustiveis e Lub.'
,'FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR':'Alimentação'
,'LOCAÇÃO DE VEÍCULOS AUTOMOTORES OU FRETAMENTO DE EMBARCAÇÕES': 'Loc. Veic. Embarcações'
,'HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.':'Hospedagem'
,'ASSINATURA DE PUBLICAÇÕES': 'Assinatura de Publicações'
,'SERVIÇO DE SEGURANÇA PRESTADO POR EMPRESA ESPECIALIZADA.':'Segurança'
,'SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO': 'Táxi, Pedágio e Estacionamento'
,'LOCAÇÃO OU FRETAMENTO DE AERONAVES':'Loc. Fret. Aeronaves'
,'PASSAGENS TERRESTRES, MARÍTIMAS OU FLUVIAIS':'Passagens (Não Aéreo)'
,'LOCAÇÃO OU FRETAMENTO DE EMBARCAÇÕES':'Loc. Emb.'
,'PARTICIPAÇÃO EM CURSO, PALESTRA OU EVENTO SIMILAR':'Cursos'
,'OUTROS':'Outros'               
}

In [ ]:
#como os gastos já estão ordenados basta selecionar os 10 primeiros
cols_to_use = reinbursements_acc.index[:10].tolist()

#para calcular o gasto percentual dividimos o gasto de cada categoria pela soma do todo
df_subquota_gb_perct = df_subquota_year.divide(df_subquota_year.sum(axis=1), axis=0)

#o gasto OUTROS é simplesmente a soma dos demais gastos além dos 10 maiores
df_subquota_gb_perct['OUTROS'] = df_subquota_gb_perct.drop(cols_to_use, axis=1).sum(axis=1)
cols_to_use = cols_to_use+['OUTROS']
df_subquota_gb_perct.head()

In [ ]:
df_subquota_gb_perct_px = df_subquota_gb_perct[cols_to_use].reset_index().melt(id_vars='year')

df_subquota_gb_perct_px.rename(columns={'subquota_description':'Categoria'
                                        , 'year':'Ano'
                                        , 'value':'Valor'}, inplace=True)


df_subquota_gb_perct_px['Categoria'] = df_subquota_gb_perct_px['Categoria'].apply(lambda x: label_dict[x])
df_subquota_gb_perct_px

In [ ]:
#Caso vocẽ queira subir o seu gráfico para o servidor do plotly basta preencher as informações de usuario
#e chave da api

#chart_studio.tools.set_credentials_file(username=<SEU_USUARIO_PLOTLY>, api_key=<CHAVE_DE_ACESSO>)

In [ ]:
layout = Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

fig = px.bar(df_subquota_gb_perct_px, x="Ano", y="Valor"
             , color="Categoria"
             , barmode = 'stack'
             , color_discrete_sequence=px.colors.qualitative.Safe
             , width=800
             , height=650
            )

fig.update_layout(layout)
fig.show()

#caso queira fazer o upload para o plotly descomente a linha abaixo
#py.iplot(fig, filename = 'evolucao_ceap', auto_open=True)

## Analise dos reembolsos por partido

> Em nossa última análise vamos observar os gastos quebrados tanto por partidos quanto por categorias. A ideia aqui é começar a traçar um perfil de gastos de cada partido. As observações feitas neste ponto poderão ser utilizadas futuramente para analises mais elaboradas

In [ ]:
#calcula os dados acumulados por partido
# Deixei essas duas variáveis para o caso de alguém querer fazer a análise acumulando um período maior
PERIODO = [2020]
FILTRO = (df_chamber_clean.year.isin(PERIODO))

df_subquota_party = df_chamber_clean[FILTRO].groupby(by=['party','year']).agg(
    sum_values = pd.NamedAgg(column='total_net_value',aggfunc=sum)
    ,max_values = pd.NamedAgg(column='total_net_value',aggfunc=max)
    ).reset_index()

#pivoteia o dataframe
df_subquota_party = df_subquota_party.pivot('party','year','sum_values')
party_year_acc = df_subquota_party.mean(axis=1).sort_values(ascending=False)/1000000

In [ ]:
party_year_acc.head()

In [ ]:
# Este dataframe vai server para acumular o período que se quira analisar
party_acc = party_year_acc.sum(axis=0,level='party').reset_index(
            name='sum_values').sort_values(by='sum_values', ascending = False)

fig,ax = plt.subplots(1,1,figsize=(20,5))

sns.barplot(data=party_acc
            , x='party'
            , y = 'sum_values'
            , ax=ax
            , palette='crest')

ax.set_xticklabels(ax.get_xticklabels()
    ,rotation=45
    ,horizontalalignment='right'
    ,fontweight='light'
    )

ax.set_ylabel("Total de Reembolsos (R$ Milhões)")
ax.set_xlabel("")


None

> Para a análise seguinte vamos agregar os dados primeiramente por cada congressista  e a seguir agregar novamente por partido. Enquanto no primeiro groupby usamos a soma dos valores no segundo usamos a média.

In [ ]:
party_year_acc_2019_2020 = df_chamber_clean[FILTRO].groupby(
            by=['party','congressperson_name','year']).agg(
                sum_values = pd.NamedAgg(column='total_net_value',aggfunc=sum)
                ).reset_index().groupby(
                                    by=['party']).agg(
                                    mean_values = pd.NamedAgg(column='sum_values',aggfunc='mean')
                                    ).reset_index()

#deixando os valores numa escala de milhar para facilitar a visualização
party_year_acc_2019_2020['mean_values'] = party_year_acc_2019_2020['mean_values']/1000

In [ ]:
party_year_acc_2019_2020.head()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(20,5))

sns.barplot(data = party_year_acc_2019_2020.sort_values(by='mean_values', ascending = False)
            , x='party'
            , y = 'mean_values'
            , ax=ax
            , palette='crest')

ax.set_xticklabels(ax.get_xticklabels()
    ,rotation=45
    ,horizontalalignment='right'
    ,fontweight='light'
    )

ax.set_ylabel("Total de Reembolsos (R$ Mil)")
ax.set_xlabel("")


### Destrinchando os gastos nas categorias para os partidos

> Primeiro vamos construir a representação do congressista médio por partido de cada partido

In [ ]:
df_subquota = df_chamber_clean[FILTRO].groupby(
        by=['party','congressperson_name','year','subquota_description']).agg(
            sum_values = pd.NamedAgg(column='total_net_value',aggfunc=sum)
            ).reset_index().groupby(
                                by=['party','subquota_description']).agg(
                                mean_values = pd.NamedAgg(column='sum_values',aggfunc='mean')
                                ).reset_index()

df_subquota_party_mean =  df_subquota.pivot('party'
                                            ,'subquota_description'
                                            ,'mean_values').reset_index()
df_subquota_party_mean.head()

In [ ]:
# Aqui eu mudei um pouco o diocionario para colocar quebras de linhas
# de forma que os nomes não ficassem tao extensos horizontalmente
label_dict = {"DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.": "Divulgação"
,"CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.": "Consultorias"
,"LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES": "Loc. Veic."
,"MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR": "Man. Escr."
,"PASSAGEM AÉREA - RPA":"Pass. Aérea\nRPA"
,"PASSAGEM AÉREA - REEMBOLSO":"Pass. Aérea\nReembolso"
,"TELEFONIA":"Telefonia"  
,'SERVIÇOS POSTAIS':'Serviços\nPostais'
,'COMBUSTÍVEIS E LUBRIFICANTES.':'Combustiveis\ne Lub.'
,'FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR':'Alimentação'
,'LOCAÇÃO DE VEÍCULOS AUTOMOTORES OU FRETAMENTO DE EMBARCAÇÕES': 'Loc. Veic.\n Embarcações'
,'HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.':'Hospedagem'
,'ASSINATURA DE PUBLICAÇÕES': 'Assinatura de Publicações'
,'SERVIÇO DE SEGURANÇA PRESTADO POR EMPRESA ESPECIALIZADA.':'Segurança\nEspecializada'
,'SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO': 'Táxi, Pedágio\nEstacionamento'
,'LOCAÇÃO OU FRETAMENTO DE AERONAVES':'Loc. Fret.\nAeronaves'
,'PASSAGENS TERRESTRES, MARÍTIMAS OU FLUVIAIS':'Passagens Terrestres\nMarítmas ou Fluviais'
,'LOCAÇÃO OU FRETAMENTO DE EMBARCAÇÕES':'Loc. Emb.'
,'PARTICIPAÇÃO EM CURSO, PALESTRA OU EVENTO SIMILAR':'Cursos'
}


> Vamos usar a função a seguir para fazer os plots. Deixando isso encapsulado podemos escolher difenretes partidos e categorias de gasto para a visualização

In [ ]:
def plot_party_vs_subquota(df_input, parties, subquotas, vertical_line = None):
    '''Esta funcao plota os gastos por partido e por diferentes categorias.
    Para cada categoria é plotado um gráfico de barras horizontais considerando 
    todos os partidos passados como parâmetro'''
    df_aux = df_input.reset_index()[['party']+subquotas]
    df_aux = df_aux[df_aux.party.isin(parties)]

    
    fig,axs = plt.subplots(1,len(subquotas),figsize=(20,7), sharey=True)
    for pos,(subq,ax) in enumerate(zip(subquotas,axs.flatten())):
        p = sns.barplot(data=df_aux, x = subq,y = 'party', ax=ax, palette='ocean')

        label = label_dict[subq] if subq in label_dict else subq
        ax.set_title(label)
        ax.set_xlabel("")
        ax.set_ylabel("")    

        if vertical_line is not None:
            ax.axvline(1, linestyle='--')
    return fig,axs

>Por fim, vamos à visualização propriamente dita. Primeiramente escolhemos as 10 categorias com os maiores gastos no período definido anteriormente e depois selecionamos os partidos de interesse

In [ ]:
COLS_TO_USE = df_chamber_clean[FILTRO].groupby(
                        'subquota_description')['total_net_value']\
                        .sum().sort_values(ascending=False).index[:10].to_list()

COLS_TO_USE = ['party']+COLS_TO_USE

PARTIDOS = ['PDT','PT','PTB', 'CIDADANIA','PV','NOVO']

#fazendo o recorte do dataframe
df_subquota_party_mean_selec = df_subquota_party_mean[
    df_subquota_party_mean.party.isin(PARTIDOS)][COLS_TO_USE]

df_subquota_party_mean_selec[COLS_TO_USE[1:]] = df_subquota_party_mean_selec[COLS_TO_USE[1:]]/1000
df_subquota_party_mean_selec.head()

In [ ]:
#Aqui estou forçando uma ordenação para casar com a ordem que vimos nos gráficos anteriores
df_subquota_party_mean_selec = df_subquota_party_mean_selec.set_index(
                                df_subquota_party_mean_selec.party).reindex(PARTIDOS)

df_subquota_party_mean_selec = df_subquota_party_mean_selec.reset_index(drop=True)

fig,ax = plot_party_vs_subquota(df_subquota_party_mean_selec
                       , PARTIDOS
                       , COLS_TO_USE[1:])
fig.text(0.5, 0.04, 'Reembolso Médio Anual por Deputado (R$ Mil)', ha='center', va='center')

#### Bônus para se divertir

>Fica o bônus abaixo para você se divertir vendo os maiores gastos em cada uma das categorias da CEAP. Já fica a dia que o deputado Carlos Bezerra provavelmente lançou um gasto de R$20.000 no lugar errado. 

In [ ]:
#maiores gastos em 2020
CATEGORIA = "FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR"
PERIODO = [2020]
df_chamber_clean[(df_chamber_clean.subquota_description==CATEGORIA)
                &(df_chamber_clean.year.isin(PERIODO))][VIEWS_COLS].sort_values(
                                                                    by='total_net_value', ascending=False)